In [7]:
import pandas as pd

id2label = pd.read_pickle('data/id2label_final.pkl')
label2id = pd.read_pickle('data/label2id_final.pkl')
submission_file = pd.read_csv('data/submission_file.csv')
test_ids_final = pd.read_csv('data/test_ids_final.csv')
test_ocr_clean = pd.read_pickle('data/test_ocr_clean.pkl')
train_ids_final = pd.read_csv('data/train_ids_final.csv')
train_labels_final = pd.read_pickle('data/train_labels_final.pkl')
train_set_ocr = pd.read_pickle('data/train_set_ocr.pkl')

In [168]:
label2id

{0: 'advertisement',
 1: 'budget',
 2: 'email',
 3: 'file_folder',
 4: 'form',
 5: 'handwritten',
 6: 'invoice',
 7: 'letter',
 8: 'memo',
 9: 'news_article',
 10: 'pit37_v1',
 11: 'pozwolenie_uzytkowanie_obiektu_budowlanego',
 12: 'presentation',
 13: 'questionnaire',
 14: 'resume',
 15: 'scientific_publication',
 16: 'scientific_report',
 17: 'specification',
 18: 'umowa_na_odleglosc_odstapienie',
 19: 'umowa_o_dzielo',
 20: 'umowa_sprzedazy_samochodu'}

In [21]:
text = test_ocr_clean['test_set/e9fdc30e-99bd-4df8-ac68-adab008dfc88.jpg']
text

'PITaxpl POLALSHPein 9220067289 8383511524 18 1 0 ofrin PIT-37 ZEZNANIE O WYSOKOSCI OSIĄGNIĘTEGO DOCHODU(P"NI9SIONEJ STRATY) roku podatkowym FoamuapnconvFoslol Oualn Alotry Goldn ua pchod[6oOcepuoronvr eryn Rrectypor Ipodlcelacecoocaomen Doauon npofconcha Pabnhoa NUb mych poonoton cgoiuchzsndach Aasosoqanual Prokadru pontonncc dadainota QOCPOda CIC Camch nuren] [lormaorOmiooatych Oodiodon lpnryhouoa) ouzoaMon soecanych proaunGIooc opodalrOuar L Gnnn to OooauriOolcrac UanhoocLodondocnoconmrcornchdaeg conuon Dochodow @sbaly (1ubeghrch Podslana Duwna At 5usL Ustawyzonn Z64pca 1991/ 0 podanu oochooonmodo  uycmnychiDz U 1 2021 r Doz 1128 Nanci dalet slawa sMadanta Od dnia 15 lulauo do dnla 30 khetniz naslepujnceoo po roku oodalkomm Zasuzezener Usiava art 45ust Lejsce sktidana Urzad korym mowa Usiawy Zwany dale] Uedem Wybor *0o,obuoco 0a . aoaoo disrosob  {oood Dradnany pmdrieny 2u1 Ustat = odru A MIEJSCE SKŁADANIA ZEZNANIA Uuad Skarbovn wNowm Sacz 0 Cclzlozonta lormularza Prhoe Rlors Mobe w

In [90]:
train = {'file1':('i want it That Way?',1),
             'file2':('TUHe may you do it!? /n come o 123-456-7890 n 93 94 2',2),
             'file3':('maybe$ you will Do IT 32 8 (123) 456-7890',1),
             'file4':('TUHe may you do it!? /n come on 93 94 2',2),
             'file5':('TUHe may you domm 123.456.7890 it!? /noo come omon 2',2),
             'file6':('TUHe may you kokodo i want it it!? /n co 123 456 7890',2),
             'file7':('okd mor uoowa o dzielo imfriofr r rwivnwroc v v',0),


      }

In [94]:
data_test = [{'file_name': key, 'string': value} for key, value in test_ocr_clean.items()]
df_test = pd.DataFrame(data_test)

In [97]:
text_vector

0       PITaxpl POLALSHPein 9220067289 8383511524 18 1...
1       Kooaek DalkoNana LiteRAV CZARNYM LLB NeBiesAV ...
2       Wypelnv Joonn Beyffenl Vazar FLENY Plsfl Delko...
3       PlTaxpl POLA JASNE WYPELN4 PoDATnIK POLA CieMN...
4       PlTaxpl POL A JASNE WypelNIA PodaTNix CienncKy...
                              ...                        
3495    Nancy Coleman 6831 Ridge Blvd. 414 BrockIyn, N...
3496    20733788/7 DEC_15.19393 40.22pM1 No.689 P.1/ T...
3497    AXXALS OF THE *EIv YORR ACADEMY OF sciecs VOLU...
3498    SENT BY :Mezz ina/Br 0wn   Inc_ 3-92 4:51PM 21...
3499                                  mal 4a Jeutl Uu + 1
Name: string, Length: 3500, dtype: object

In [109]:
pattern = r"\battachments\b"
target_phrase = 'attachments'

text_vector = df_test['string']

s = text_vector.apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
f = text_vector.apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)


print('basic:', s.sum())
print('fuzz:', f.sum())

basic: 20
fuzz: 21


In [91]:
data = [{'file_name': key, 'string': value[0], 'id': value[1]} for key, value in train.items()]
df = pd.DataFrame(data)
df

,file_name,string,id
0,file1,i want it That Way?,1
1,file2,TUHe may you do it!? /n come o 123-456-7890 n ...,2
2,file3,maybe$ you will Do IT 32 8 (123) 456-7890,1
3,file4,TUHe may you do it!? /n come on 93 94 2,2
4,file5,TUHe may you domm 123.456.7890 it!? /noo come ...,2
5,file6,TUHe may you kokodo i want it it!? /n co 123 4...,2
6,file7,okd mor uoowa o dzielo imfriofr r rwivnwroc v v,0


In [92]:
import re
from fuzzywuzzy import fuzz

df['string_length'] = df['string'].apply(len)

# Add a column for the count of capital letters
df['capital_letters_count'] = df['string'].apply(lambda x: sum(1 for c in x if c.isupper()))

# Add a column for the count of numbers
df['numbers_count'] = df['string'].apply(lambda x: sum(1 for c in x if c.isdigit()))
# Add a column for the count of question marks
df['question_marks_count'] = df['string'].apply(lambda x: x.count('?'))
# Add a column for the count of currency signs
df['currency_signs_count'] = df['string'].apply(lambda x: x.count('$') + x.count('€') + x.count('£'))

# american phone regex
pattern = r"\b\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b"

# american address regex
pattern1 = r"^\d+\s+([a-zA-Z]+\s+)+[a-zA-Z]+,\s*[A-Za-z]{2}\s*\d{5}(?:-\d{4})?$"

# Add a column for the flag indicating pattern match
df['flag_phone'] = df['string'].apply(lambda x: 1 if re.search(pattern, x) else 0)

# Define the pattern to search for
pattern = r"\bi want it\b"

# Add a column for the flag indicating pattern match
df['flag_text'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)

target_phrase = "umowa o dzieło"

# Add a column for the flag indicating approximate match
df['flag_sentance_with_ratio'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)


df

,file_name,string,id,string_length,capital_letters_count,numbers_count,question_marks_count,currency_signs_count,flag_phone,flag_text,flag_sentance_with_ratio
0,file1,i want it That Way?,1,19,2,0,1,0,0,1,0
1,file2,TUHe may you do it!? /n come o 123-456-7890 n ...,2,53,3,15,1,0,1,0,0
2,file3,maybe$ you will Do IT 32 8 (123) 456-7890,1,41,3,13,0,1,1,0,0
3,file4,TUHe may you do it!? /n come on 93 94 2,2,39,3,5,1,0,0,0,0
4,file5,TUHe may you domm 123.456.7890 it!? /noo come ...,2,52,3,11,1,0,1,0,0
5,file6,TUHe may you kokodo i want it it!? /n co 123 4...,2,53,3,10,1,0,1,1,0
6,file7,okd mor uoowa o dzielo imfriofr r rwivnwroc v v,0,47,0,0,0,0,0,0,1


## Model for Polish Labels

In [420]:
train = pd.read_csv('train_data_v2.csv')
test = pd.read_csv('test_data_v2.csv')

In [421]:
train.head()

,Unnamed: 0,File,Text ING,ID,Text Adam,Nrow,Text Adam pl,isPolish,isEnglish,isOther
0,170,8975a2c2-f0a4-4830-9095-43143e1fdc32,PlTaxpl POLA JASNEWYPELRIA pooainir PolaCieVEw...,10,| W c wc ECCE GU Pirax.pl Pa a ir POK POŁACIE ...,54,| W c wc ECCE GU Pirax.pl Pa a ir POK POLACIE ...,1,0,0
1,156,10a30e16-bdd0-42ba-9b0d-256c57e36730,PlTaxpl PCLAJasneWypelna FodatniK PdlaceMeWYPe...,10,AG r jc ai ania 15 jl 22059 PITRDKĄI rowe rada...,51,AG r jc ai ania 15 jl 22059 PITRDKAI rowe rada...,1,0,0
2,189,e0af7e58-2804-4991-87d7-028326cff3ca,PITaxpl 3676752813 1498024552 PIT-37 ZEZNANIE ...,10,+8 ©. PITdYpl sea no wawa wocapao ocen wa ni b...,40,+8 ©. PITdYpl sea no wawa wocapao ocen wa ni b...,1,0,0
3,293,59005f09-95b2-41d1-8c81-6ccf2d3b34ed,PolA JASNe WypelNA PODATMIK Cieln( JoF PESEL U...,10,© PITA.pl ro use weta Poor. PoA crime WYPELA U...,51,© PITA.pl ro use weta Poor. PoA crime WYPELA U...,1,0,0
4,184,eb4c0148-fc4d-42ef-97ab-32088ca56e40,CraRnymLUO NitHESKIM KDLCREM GiemnewYpeln Urza...,10,"ZE FS JĄ , Naluiinaraaui mma mażama M ma a SR ...",52,"ZE FS JA , Naluiinaraaui mma mazama M ma a SR ...",1,0,0


In [422]:
train = train[train['isPolish']==1]
test = test[test['isPolish']==1]

In [423]:
import copy

def create_df(df1,colname):
    df = df1.copy()
    df['string'] = df[colname]
    df['string_length'] = df['string'].apply(len)
    df['capital_letters_ratio'] = df['string'].apply(lambda x: sum(1 for c in x if c.isupper()))
    df['capital_letters_ratio'] = df['capital_letters_ratio'] /df['string_length']
    df['numbers_count'] = df['string'].apply(lambda x: sum(1 for c in x if c.isdigit()))
    df['question_marks_count'] = df['string'].apply(lambda x: x.count('?'))
    df['currency_signs_count'] = df['string'].apply(lambda x: x.count('$') + x.count('€') + x.count('£'))

    pattern = r"\bumowa sprzedazy samochodu\b"
    df['flag_1'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
    target_phrase = "umowa sprzedazy samochodu"
    df['flag_fuzz1'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)
    
    pattern = r"\bumowa o dzielo\b"
    df['flag_2'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
    target_phrase = "umowa o dzielo"
    df['flag_fuzz2'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)
    
    pattern = r"\boswiadczenie o odstapieniu od umowy zawartej na odleglosc\b"
    df['flag_3'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
    target_phrase = "oswiadczenie o odstapieniu od umowy zawartej na odleglosc"
    df['flag_fuzz3'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)
    
    pattern = r"\bna podstawie art. 7 ust. 1 ustawy\b"
    df['flag_4'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
    target_phrase = "na podstawie art. 7 ust. 1 ustawy"
    df['flag_fuzz4'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)
    
    pattern = r"\bwniosek o udzielenie pozwolenia na uzytkowanie obiektu budowlanego\b"
    df['flag_5'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
    target_phrase = "wniosek o udzielenie pozwolenia na uzytkowanie obiektu budowlanego"
    df['flag_fuzz5'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)
    
    pattern = r"\bzeznanie o wysokosci osiagnietego dochodu\b"
    df['flag_6'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
    target_phrase = "zeznanie o wysokosci osiagnietego dochodu"
    df['flag_fuzz6'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)
    
    pattern = r"\bPitaxpl\b"
    df['flag_7'] = df['string'].apply(lambda x: 1 if re.search(pattern, x, re.IGNORECASE) else 0)
    target_phrase = "Pitaxpl"
    df['flag_fuzz7'] = df['string'].apply(lambda x: 1 if fuzz.partial_ratio(target_phrase.lower(), x.lower()) > 70 else 0)
    return df

In [471]:
df = create_df(train,'Text ING')
pd.unique(df['ID'])

array([10, 11, 18, 19, 20,  0, 17,  3,  5,  9,  6,  1])

In [476]:
is_good = np.zeros(len(df['ID']))
i=0
for ids in df['ID']:
    if (ids in (10,11,18,19,20)):
        is_good[i] = 1
    i=i+1
 


indices = np.where(np.array(is_good) == 1)[0]
df = df.iloc[indices]
X = df.drop(columns=['Text Adam','Unnamed: 0','string','File','Text ING','ID','Text Adam pl','isPolish','isEnglish','isOther'])
y = df['ID']

In [477]:
X.columns

Index(['Nrow', 'string_length', 'capital_letters_ratio', 'numbers_count',
       'question_marks_count', 'currency_signs_count', 'flag_1', 'flag_fuzz1',
       'flag_2', 'flag_fuzz2', 'flag_3', 'flag_fuzz3', 'flag_4', 'flag_fuzz4',
       'flag_5', 'flag_fuzz5', 'flag_6', 'flag_fuzz6', 'flag_7', 'flag_fuzz7'],
      dtype='object')

In [478]:
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score



# Create a DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state = 30)

# Perform cross-validation with 5 folds
scores = cross_val_score(clf, X, y, cv=5)

clf.fit(X,y)

# Print the cross-validation scores
print("Cross-validation scores:", scores)
print("Average accuracy:", scores.mean())

Cross-validation scores: [1.         0.89189189 0.91891892 0.91891892 0.94594595]
Average accuracy: 0.9351351351351351


In [479]:
df = create_df(test,'Text ING')
X_val = df.drop(columns=['Text Adam','Unnamed: 0','string','File','Text ING','ID','Text Adam pl','isPolish','isEnglish','isOther'])
y_val = df['ID']
y_pred_val= clf.predict(X_val)

accuracy_score(y_val, y_pred_val)

0.9893048128342246

In [480]:
pd.Series(y_val != y_pred_val).groupby(y_val).sum()


ID
5     1
9     1
10    0
11    0
18    0
19    0
20    0
Name: ID, dtype: int64

In [481]:
import joblib

# Save the trained model to a file
joblib.dump(clf, 'clf_model.pkl')

['clf_model.pkl']